## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [8]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [2]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([28*28, 128]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([128]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([128, 10]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, [-1, 28*28])  # Flatten the input
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)  # Hidden layer with ReLU activation
        logits = tf.matmul(h1, self.W2) + self.b2  # Output layer
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [3]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 8.727601 ; accuracy 0.6961
epoch 1 : loss 8.710278 ; accuracy 0.69655
epoch 2 : loss 8.693051 ; accuracy 0.6969333
epoch 3 : loss 8.675921 ; accuracy 0.69738334
epoch 4 : loss 8.658887 ; accuracy 0.69795
epoch 5 : loss 8.641948 ; accuracy 0.6983333
epoch 6 : loss 8.625106 ; accuracy 0.69876665
epoch 7 : loss 8.608359 ; accuracy 0.69916666
epoch 8 : loss 8.591712 ; accuracy 0.69955
epoch 9 : loss 8.575159 ; accuracy 0.69991666
epoch 10 : loss 8.558698 ; accuracy 0.70035
epoch 11 : loss 8.542326 ; accuracy 0.7007167
epoch 12 : loss 8.526041 ; accuracy 0.7011833
epoch 13 : loss 8.509845 ; accuracy 0.7016
epoch 14 : loss 8.493735 ; accuracy 0.7019167
epoch 15 : loss 8.477712 ; accuracy 0.70225
epoch 16 : loss 8.461773 ; accuracy 0.7026167
epoch 17 : loss 8.445921 ; accuracy 0.7029667
epoch 18 : loss 8.430153 ; accuracy 0.7034
epoch 19 : loss 8.414471 ; accuracy 0.7037
epoch 20 : loss 8.398873 ; accuracy 0.7040667
epoch 21 : loss 8.383359 ; accuracy 0.70445
epoch 22 : loss 8.